In [1]:
import pathlib
import sys
import yaml
import re

sys.path.append("./../")
sys.path.append("./../../../ParallelWaveGAN/")

import time
import torch
import numpy as np
import matplotlib.pyplot as plt

from TTS.tts.utils.generic_utils import setup_model
from TTS.tts.utils.text import text_to_sequence, phoneme_to_sequence
from TTS.tts.utils.text.symbols import make_symbols, phonemes, symbols
from TTS.utils.audio import AudioProcessor
from TTS.utils.io import load_config, AttrDict
from TTS.tts.utils.synthesis import synthesis

import parallel_wavegan.models


from parallel_wavegan.datasets import MelDataset
from parallel_wavegan.datasets import MelSCPDataset
from parallel_wavegan.utils import read_hdf5

from IPython.display import Audio, display

In [2]:
config_path = pathlib.Path("/home/kwantics/python3-tasks/mozillaTTS/TriptiCloudForwardGravesAttention/hrs33hop256HindiOnly/fastspeech2/gpu_machine_models/RMSLE_post_net_decoder_inc/config.json")
model_path = pathlib.Path("/home/kwantics/python3-tasks/mozillaTTS/TriptiCloudForwardGravesAttention/hrs33hop256HindiOnly/fastspeech2/gpu_machine_models/RMSLE_post_net_decoder_inc/checkpoint_65000.pth.tar")

In [3]:
vocoderModelPath = './../../../ParallelWaveGAN/models/fixedAp/configOnline/checkpoint-1200000steps.pkl'
vocoderConfig    = './../../../ParallelWaveGAN/models/fixedAp/configOnline/config.yml'


In [4]:
use_cuda=False

In [5]:
config = load_config(str(config_path))

## Load pwgan

In [6]:
def loadPwGan(vocoderConfig, vocoderModelPath):
    # initialize WaveRNN

    vocoderConfigFile = open(vocoderConfig)
    vconfig = yaml.load(vocoderConfigFile, Loader=yaml.Loader)


    if torch.cuda.is_available():
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")
    model_class = getattr(
        parallel_wavegan.models,
        vconfig.get("generator_type", "ParallelWaveGANGenerator"))
    vocoderModel = model_class(**vconfig["generator_params"])
    vocoderModel.load_state_dict(
        torch.load(vocoderModelPath, map_location="cpu")["model"]["generator"])

    vocoderModel.remove_weight_norm()
    vocoderModel = vocoderModel.eval().to(device)
    vConfigTTS = AttrDict()
    vConfigTTS.update(vconfig)
    apVocoder = AudioProcessor(**vConfigTTS.audio)
    
    return vocoderModel,vconfig,apVocoder



def fromParallelGan(c,config,model):
    
    if torch.cuda.is_available():
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")
    use_noise_input = not isinstance(
        model, parallel_wavegan.models.MelGANGenerator)
    pad_fn = torch.nn.ReplicationPad1d(
        config["generator_params"].get("aux_context_window", 0))
    x = ()
    if use_noise_input:
        z = torch.randn(1, 1, len(c) * config["hop_size"]).to(device)
        x += (z,)
    c = pad_fn(torch.from_numpy(c).unsqueeze(0).transpose(2, 1)).to(device)
    x += (c,)
    y = model.inference(*x).view(-1).cpu().detach().numpy()
    return y

In [7]:
if 'characters' in config.keys():
    symbols, phonemes = make_symbols(**config.characters)
    
ap_synth = AudioProcessor(**config.audio)
num_chars = len(phonemes) if config.use_phonemes else len(symbols)
num_speakers = 0

model = setup_model(num_chars, num_speakers, config)

state_dict = torch.load(model_path)

 > Setting up Audio Processor...
 | > sample_rate:22050
 | > num_mels:80
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > stats_path:None
 | > hop_length:256
 | > win_length:1024
 > Using model: Fastspeech2


In [8]:
model.load_state_dict(state_dict["model"])

<All keys matched successfully>

In [9]:
model.eval()
model = model.cuda()

In [10]:
# load vocoder
vocoderModel,vconfig,apVocoder = loadPwGan(vocoderConfig, vocoderModelPath)

 > Setting up Audio Processor...
 | > sample_rate:22050
 | > num_mels:80
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > stats_path:None
 | > hop_length:256
 | > win_length:1024


In [11]:
def text_to_seqvec(text, CONFIG):
    text_cleaner = [CONFIG.text_cleaner]
    # text ot phonemes to sequence vector
    if CONFIG.use_phonemes:
        seq = np.asarray(
            phoneme_to_sequence(text, text_cleaner, CONFIG.phoneme_language,
                                CONFIG.enable_eos_bos_chars,
                                tp=CONFIG.characters if 'characters' in CONFIG.keys() else None),
            dtype=np.int32)
    else:
        seq = np.asarray(text_to_sequence(text, text_cleaner, tp=CONFIG.characters if 'characters' in CONFIG.keys() else None), dtype=np.int32)
    return seq

In [12]:
import requests
import json
import string
import re

url     = 'https://kwantics.ai/pyApis/transliterate'
modelTrans   = "eng2hindi" # Values are eng2hindi and hindi2eng
headers = {
'Accept': 'application/json, text/plain, */*',
'Accept-Encoding': 'gzip, deflate, br',
'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8',
'Connection': 'keep-alive',
'Content-Length': '41',
'Content-Type': 'undefined',
'Host': 'dev.kwantics.ai',
'Origin': 'https://kwantics.ai',
'Referer':"https://kwantics.ai/apps/utility/",
'Sec-Fetch-Mode': 'cors',
"Sec-Fetch-Site": "same-origin"
}
def fetchTransliteration(word,model):
    data = {"model":model,"word":word}
    json_data = None
    while not json_data:
        try:
            res  = requests.post(url = url, data = json.dumps(data),headers=headers)
            json_data = json.loads(res.text)
        except Exception as e:
            json_data = None
            pass
    return json_data['transliteration'][0]

def transliterateSentence(sent, model):
    transliteratedList = []
    languageRegex = re.compile("[\u0900-\u097F]") if model == "hindi2eng" else re.compile("[a-zA-Z]")
    for word in sent.split():
        if not languageRegex.search(word) or word in string.punctuation:
            transliteratedList.append(word)
        else:
            transliteratedList.append(fetchTransliteration(word,model))
    return " ".join(transliteratedList)

In [13]:
def tts(config, model, use_cuda, text, alpha_speed, alpha_pitch, alpha_energy):
    # Convert to tensors and things
    inputs = text_to_seqvec(text, config)
    inputs = torch.as_tensor(inputs, dtype=torch.long)
    inputs = inputs.cuda()
    inputs = inputs.unsqueeze(0)
    
    # inference model
    return model.inference(inputs, alpha_speed=alpha_speed, alpha_pitch=alpha_pitch, alpha_energy=alpha_energy)

In [14]:
long_text = """
एक किसान था. उसके पाँच बेटे थे. सभी बलवान और मेहनती थे. पर वे हमेशा आपस में लड़ते झगड़ते रहते थे. किसान यह देख कर बहुत चिंतित रहा करता था. वह चाहता था कि उसके बेटे आपस में लड़ाई झगड़ा न करें और मेलजोल से रहें. किसान ने अपने बेटों को बहुत समझाया और डाँटा फटकारा भी, पर उनपर इसका कोई असर नहीं हुआ.

किसान को हमेशा यही चिंता सताती रहती कि वह अपने बेटों में एकता कैसे कायम करे! एक दिन उसे अपनी समस्या का एक उपाय सूझा. उसने अपने पाँचों बेटों को बुलाया. उन्हें लकडि़यों का एक गट्ठर दिखाकर उसने पूछाँ, "क्या तुममें से कोई इस गट्ठर को खोले बिना तोड़ सकता है?"

किसान के पाँचों बेटे बारी बारी से आगे आए. उन्होंने खूब ताकत लगाई. पर उनमें से कोई भी लकडि़यों का गट्ठर तोड़ नही सका.

फिर किसान ने गट्ठर खोलकर लकडि़यों को अलग अलग कर दिया. उसने अपने बेटों को एक एक लकड़ी देकर उसे तोड़ने के लिए कहा. सभी लड़कों ने बहुत आसानी से अपनी अपनी लकडी तोड़ डाली.

किसान ने कहा,"देखा! एक एक लकड़ी को तोड़ना कितना आसान होता है. इन्हीं लकडि़यों को एक साथ गट्ठर में बाँध देने पर ये कितनी मजबूत हो जाती हैं. इसी तरह तुम लोग मिल जुल कर एक साथ रहोगे, तो मजबूत बनोगे और लड़ झगड़कर अलग अलग हो जाओगे, तो कमजोर बनोगे."

शिक्षा एकता में ही शक्ति है, फूट में ही है विनाश.
"""

long_text2 = """
हो मजनू का दिल लंदन में था और लैला थी बिहार की लैला थी बिहार की हो दोनों के मिलने को ना थी परमिशन ही सरकार की हो मजनू का दिल लंदन में था और लैला थी बिहार की हो दोनों के मिलने को ना थी परमिशन ही सरकार की बड़ी तेज़ मोहब्बत जाग गयी पीछे के रास्ते भाग गई अरे आग लगा गयी आग लगा गयी गांव में पूरे प्यार की मेरी जान मेरी जान मेरी जान ऐसी तैसी हो गयी संस्कार की मेरी जान ऐसी तैसी हो गयी संस्कार की मेरी जान ऐसी तैसी हो गयी संस्कार की हां तब तक जीने का क्या मतलब जब तक ना होवें शादी घर वालों के अप्रूवल पे ही कैद है आज़ादी जो कहके शादी हो कह दो इग्नोर करो इत्यादि अपनी भी ज़िम्मेदारी है हिन्दी ट्रैक्स डॉट इन इस देश की बढ़े आबादी लैला को लेकिन चैन कहाँ दिल पे दुनिया का बेन कहाँ मम्मी पापा से बिन पूछे फैमिली प्लानिंग तैयार की मेरी जान मेरी जान मेरी जान ऐसी तैसी हो गयी संस्कार की मेरी जान ऐसी तैसी हो गयी संस्कार की मेरी जान ऐसी तैसी हो गयी संस्कार की
"""

In [47]:
text = "कॉन्सेक्रेटेड टु द संस्कृत महारिशी पनीनी, ऑल्सो नॉट ऐन ऑर्डिन्री लिंग्विस्ट, आर्किटेक्ट ऑव महारिशी, महारिशी कट्यान, ऐंड योगा साइंस इज़, पनहिजिल."
text2 = 'यह आसान सा sentence मैं अभी भी unable to speak हूँ , things are getting काफी मुश्किल .'
alpha_pitch = 1.2
alpha_speed = 1.0
alpha_energy = 1.0
kwars = {
            "alpha_pitch" : alpha_pitch,
            "alpha_speed" : alpha_speed,
            "alpha_energy" : alpha_energy
        }

modelTrans = "eng2hindi"

In [48]:
t1 = time.perf_counter()
transliteratedText = transliterateSentence(text2, modelTrans)
print (transliteratedText)
print (f"Transliteration time > {time.perf_counter()-t1}")

post_mels, mels, duration, pich_energy_tup = tts(config, model, use_cuda, transliteratedText, **kwars)
print (f"Synthesis time is: {time.perf_counter()-t1}")
mels = mels[0].detach().data.cpu().numpy()
post_mels_numpy = post_mels.detach().cpu().squeeze(0).numpy()

यह आसान सा सेंटेंस मैं अभी भी अनेबल टू स्पीक हूँ , थिंग्ज़ आर गेटिंग काफी मुश्किल .
Transliteration time > 1.0779079149997415
tensor([82], dtype=torch.int32) torch.Size([1, 82])
Synthesis time is: 1.3755008809994251


t1 = time.perf_counter()
wav = ap_synth.inv_melspectrogram(post_mels_numpy.T)
print (f"Synthesis time is: {time.perf_counter()-t1}")
Audio(wav,rate= ap_synth.sample_rate)

In [49]:
t1 = time.perf_counter()
pwGanAudio                     = fromParallelGan(post_mels_numpy, vconfig, vocoderModel)
print (f"Synthesis time is: {time.perf_counter()-t1}")

Synthesis time is: 0.7205849530000705


In [50]:
Audio(pwGanAudio,rate= ap_synth.sample_rate)

In [52]:
import noisereduce as nr
audioRef = np.load("noise_fastspeech2.npy")
reduced_noise = nr.reduce_noise(audio_clip=pwGanAudio, noise_clip=audioRef,
                                n_grad_time=4,n_grad_freq=2,
                                n_std_thresh=5, prop_decrease=1,verbose=False)
Audio(reduced_noise,rate=ap_synth.sample_rate)

In [125]:
# Sample noise

start_sec = 3.8
end_sec = 4.25
indexes      = int(start_sec*ap_synth.sample_rate),int(end_sec*ap_synth.sample_rate)
audioRef     = pwGanAudio[indexes[0]:indexes[1]]

In [126]:
Audio(audioRef, rate = ap_synth.sample_rate)

In [127]:
#np.save("noise_fastspeech2.npy", audioRef)